In [1]:
import argparse
import collections
import torch
import numpy as np
import model.loss as module_loss
import model.metric as module_metric
from parse_config import ConfigParser
from utils.util import create_model, create_dataloader, create_trainer


In [2]:
# fix random seeds for reproducibility
SEED = 125
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [3]:
args = argparse.ArgumentParser(description='Emotion Reasoning in Daily Life')
args.add_argument('-c', '--config', default='train.json', type=str,
                  help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                  help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default="0", type=str,
                  help='indices of GPUs to enable (default: all)')

# custom cli options to modify configuration from default values given in json file.
CustomArgs = collections.namedtuple('CustomArgs', 'flags type target')
options = [
    CustomArgs(['--lr', '--learning_rate'], type=float, target='optimizer;args;lr'),
    CustomArgs(['--bs', '--batch_size'], type=int, target='data_loader;args;batch_size')
]

In [4]:
config = ConfigParser.from_args(args, options)

jupyter


In [5]:
config._config['name'] += "_add_text"

In [6]:
print(config._config)

OrderedDict([('name', 'Train_9_2022_add_text'), ('message', 'MEmoR Baseline'), ('anno_file', 'data/anno.json'), ('data_file', 'data/data.json'), ('id_file', 'data/train_id.txt'), ('emo_type', 'primary'), ('speakers', ['sheldon', 'leonard', 'howard', 'rajesh', 'penny', 'bernadette', 'amy', 'others']), ('audio', OrderedDict([('feature_file', 'data/features/audio_features.json'), ('feature_dim', 6373)])), ('text', OrderedDict([('bert_model_name', 'bert-large-uncased'), ('feature_file', 'data/features/text_features.json'), ('feature_dim', 1024)])), ('visual', OrderedDict([('frame_path', 'data/clips_frames'), ('faces_feature_dir', 'data/features/visual_features/face_features'), ('faces_names_dir', 'data/features/visual_features/face_names'), ('obj_feature_dir', 'data/features/visual_features/object_features'), ('env_feature_dir', 'data/features/visual_features/environment_features'), ('dim_env', 2048), ('dim_obj', 1230), ('dim_face', 1024)])), ('personality', OrderedDict([('used', True), ('

In [7]:
logger = config.get_logger('train')

In [8]:
from data_loader_add.data_loaders_add import AddMEmoRDataLoader

In [9]:
# setup data_loader instances
data_loader = AddMEmoRDataLoader(config)
valid_data_loader = data_loader.split_validation()

Initializing VisualFeatureExtractor...
Initializing AudioFeatureExtracor...
Initializing TextFeatureExtractor...
Initializing PersonalityFeatureExtractor...
Initializing Add_TextFeatureExtractor...


  0%|▏                                                                                                                       | 12/6829 [00:00<01:06, 103.12it/s]

vectorize features.....


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6829/6829 [00:47<00:00, 144.24it/s]


In [10]:
from model_add.model_add import AddAMER

In [11]:
# build model architecture, then print to console
model = AddAMER(config)

In [12]:
# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
metrics = [getattr(module_metric, met) for met in config['metrics']]

In [13]:
from trainer_add import AddMEmoRTrainer

In [14]:
trainer = AddMEmoRTrainer(model, criterion, metrics,
                      config=config,
                      data_loader=data_loader,
                      valid_data_loader=valid_data_loader,
                    )      

In [15]:
print(next(trainer.model.parameters()).is_cuda)

True


In [16]:
trainer.train()

Train Epoch: 1 [0/5464 (0%)] Loss: 2.172423 Time:2022-03-20 22:19:18
Train Epoch: 1 [4800/5464 (88%)] Loss: 1.484509 Time:2022-03-20 22:20:53
    epoch          : 1
    loss           : 1.8949200584177386
    accuracy       : 0.31548582995951385
    macro_f1       : 0.2480224461523265
    weighted_f1    : 0.2762939593860873
    val_loss       : 2.0450972172251918
    val_accuracy   : 0.24835164835164836
    val_macro_f1   : 0.20641468278572775
    val_weighted_f1: 0.24967010805720882
Saving current best: model_best.pth ...
Current best val_accuracy: 0.24835164835164836 at epoch 1, dir: 0320_221756
Train Epoch: 2 [0/5464 (0%)] Loss: 1.354020 Time:2022-03-20 22:21:44
Train Epoch: 2 [4800/5464 (88%)] Loss: 1.093975 Time:2022-03-20 22:23:18
    epoch          : 2
    loss           : 1.2236476113921717
    accuracy       : 0.5853913630229421
    macro_f1       : 0.5264332526496516
    weighted_f1    : 0.5741222391538437
    val_loss       : 1.8913168635284692
    val_accuracy   : 0.3641025

In [17]:
exit()

In [18]:
target, U_v, U_a, U_t, U_p, M_v, M_a, M_t, target_loc, umask, seg_len, n_c, U_t_add = next(iter(data_loader))

In [19]:
U_t.shape

torch.Size([24, 36, 1024])

In [20]:
U_t

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0311,  0.0591, -0.2860,  ..., -0.4797, -0.3555, -0.2076],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.3857,  0.1044, -0.5213,  ..., -0.2447,  0.1187,  0.0634],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.1687, -0.3514, -0.4359,  ..., -0

In [21]:
U_t_add.shape

torch.Size([24, 36, 24])

In [22]:
import torch.nn as nn

In [23]:
enc_t_add = nn.Sequential(
            nn.Linear(24, 128 * 4),
            nn.ReLU(),
            nn.Linear(128 * 4, int(0.5 * 128)), # D_e = 128
        )    